In [1]:
import tensorflow as tf
from tensorflow import keras
import json

# InceptionNet

In [2]:
inception_v3 = keras.applications.InceptionV3()
inception_v3.summary()

Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 299, 299, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 149, 149, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 149, 149, 32)      │              96 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 149, 149, 32)      │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 147, 147, 32)      │           9,216 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 147, 147, 32)      │              96 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 147, 147, 32)      │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 147, 147, 64)      │          18,432 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 147, 147, 64)      │             192 │ conv2d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 147, 147, 64)      │               0 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 73, 73, 64)        │               0 │ activation_2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 73, 73, 80)        │           5,120 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 73, 73, 80)        │             240 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_3 (Activation)     │ (None, 73, 73, 80)        │               0 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 71, 71, 192)       │         138,240 │ activation_3[0][0]         │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 23,851,784 (90.99 MB)

 Trainable params: 23,817,352 (90.86 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [7]:
len(inception_v3.layers)

313

In [3]:
def model(input_shape, n_classes):
    # Drop the last layer
    base_model = keras.applications.InceptionV3(
        input_shape=input_shape,
        include_top=False
    )

    # Freeze the base model by making it non-trainable
    base_model.trainable = False

    # Create the input layer
    inputs = keras.layers.Input(shape=input_shape)
    scaled_inputs = keras.applications.inception_v3.preprocess_input(inputs)

    # Set training to False to avoid keeping track of statistics in the batch norm layer
    output_of_baseModel = base_model(scaled_inputs, training=False)  # output is 3D

    # Global Average Pooling turns each channel into a scalar value. No need to flatten
    A0 = keras.layers.GlobalAveragePooling2D()(output_of_baseModel)
    dropOut = keras.layers.Dropout(0.2)(A0)
    outputs = keras.layers.Dense(units=n_classes)(dropOut)

    model = keras.models.Model(inputs, outputs)
    return model

In [4]:
INPUT_SHAPE = (160, 160, 3)
N_FEATURES = 128

model = model(INPUT_SHAPE, N_FEATURES)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 160, 160, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ true_divide (TrueDivide)             │ (None, 160, 160, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ subtract (Subtract)                  │ (None, 160, 160, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_v3 (Functional)            │ (None, 3, 3, 2048)          │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         262,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,065,056 (84.17 MB)

 Trainable params: 262,272 (1.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [8]:
inception_v3.load_weights('keras-facenet-h5/model.h5')

ValueError: Layer count mismatch when loading weights from file. Model expected 189 layers, found 245 saved layers.

# Triplet Loss

For an image $x$, its encoding is denoted as $f(x)$, where $f$ is the function computed by the neural network.

<div style="text-align: center;">
    <img src="images/f_x.png" style="width:400px;height:200px;">
</div>

### Triplet Training:
- Training will use **triplets of images** $(A, P, N)$, where:
    - **A** is the "Anchor" image — a picture of a person.
    - **P** is the "Positive" image — a picture of the same person as the Anchor.
    - **N** is the "Negative" image — a picture of a different person than the Anchor.

- These triplets are selected from the training dataset. Let $(A^{(i)}, P^{(i)}, N^{(i)})$ denote the $i$-th training example.

- You aim to ensure that an image $A^{(i)}$ (Anchor) is **closer** to the Positive $P^{(i)}$ than to the Negative $N^{(i)}$, by at least a margin $\alpha$:

$$
|| f\left(A^{(i)}\right)-f\left(P^{(i)}\right)||_{2}^{2}+\alpha<|| f\left(A^{(i)}\right)-f\left(N^{(i)}\right)||_{2}^{2}
$$

### Triplet Loss:
- We define the following **triplet loss** function:

$$
\mathcal{J} = \sum^{m}_{i=1} \left[ \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \right]_+
\tag{3}
$$

> **Note**: The notation "$[z]_+$" denotes $\max(z, 0)$.

### Key Points:
- Term (1): Squared distance between the **anchor** (A) and the **positive** (P) for a given triplet; you want this to be small.
- Term (2): Squared distance between the **anchor** (A) and the **negative** (N) for a given triplet; you want this to be large
- **$\alpha$**: This is the margin—a manually chosen hyperparameter. Here, $\alpha = 0.2$.

> **Note**: In most implementations, the encoding vectors are rescaled to have an L2 norm equal to 1 (i.e., $\mid \mid f(img) \mid \mid_2 = 1$). You won’t need to handle this here.


In [5]:
def triplet_loss(Y_true, Y_pred, alpha=0.2):
    anchor, positive, negative = Y_pred[0], Y_pred[1], Y_pred[2]

    pos_dist = tf.subtract(anchor, positive)  # (None, n_features)
    pos_dist = tf.square(pos_dist)  # (None, n_features)
    pos_dist = tf.reduce_sum(pos_dist, axis=-1)  # (None, )

    neg_dist = tf.subtract(anchor, negative)  # (None, n_features)
    neg_dist = tf.square(neg_dist)  # (None, n_features)
    neg_dist = tf.reduce_sum(neg_dist, axis=-1)  #  (None, )

    loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)  # (None, )
    loss = tf.maximum(loss, 0)  # (None, )
    loss = tf.reduce_sum(loss)
    return loss

# Dataset